# Load Libraries and Model

In [1]:
import numpy as np
import tensorflow as tf
import tensorflow.keras as keras 
import nibabel as nib
import os
import glob
from tqdm import tqdm
from keras.models import load_model
my_model = load_model('C:/Users/Fungj/Documents/EECE_571F/unet_model_20220401.h5', 
                      compile=False)

In [2]:
def crop_3D(img, new_size):
    img_shape = img.shape
    x_mid = int(img_shape[0]/2)
    y_mid = int(img_shape[1]/2)
    z_mid = int(img_shape[2]/2)

    x_diff = int(abs(new_size[0]-x_mid))
    y_diff = int(abs(new_size[1]-y_mid))
    z_diff = int(abs(new_size[2]-z_mid))

    x_start = x_mid-x_diff
    y_start = y_mid-y_diff
    z_start = z_mid-z_diff

    tmp_img = img[x_start:x_start+new_size[0],y_start:y_start+new_size[1],z_start:z_start+new_size[2]]
    return tmp_img

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

def test_generate_brats_batch(file_pattern, 
                         contrasts, 
                         batch_size=32, 
                         tumour='*', 
                         patient_ids='*',
                         crop_size = (None,None,None), 
                         augment_size=None,
                         infinite = True):
    """
    Generate arrays for each batch, for x (data) and y (labels), where the contrast is treated like a colour channel.
    
    Example:
    x_batch shape: (32, 240, 240, 155, 4)
    y_batch shape: (32, 240, 240, 155)
    
    augment_size must be less than or equal to the batch_size, if None will not augment.
    
    """
    while True:
        n_classes = 4

        # get list of filenames for every contrast available
        keys = dict(prefix=prefix, tumour=tumour)
        filenames_by_contrast = {}
        for contrast in contrasts:
            filenames_by_contrast[contrast] = glob.glob(file_pattern.format(contrast=contrast, patient_id=patient_ids, **keys)) if patient_ids == '*' else []
            if patient_ids != '*':
                contrast_files = []
                for patient_id in patient_ids:
                    contrast_files.extend(glob.glob(file_pattern.format(contrast=contrast, patient_id=patient_id, **keys)))
                filenames_by_contrast[contrast] = contrast_files

        # get the shape of one 3D volume and initialize the batch lists
        arbitrary_contrast = contrasts[0]
        if crop_size == (None,None,None):
            shape = nib.load(filenames_by_contrast[arbitrary_contrast][0]).get_fdata().shape
        else:
            shape = crop_size

        # initialize empty array of batches
        x_batch = np.empty((batch_size, ) + shape + (len(contrasts), )) #, dtype=np.int32)
        y_batch = np.empty((batch_size, ) + shape + (n_classes,)) #, dtype=np.int32)
        num_images = len(filenames_by_contrast[arbitrary_contrast])
#         np.random.shuffle(filenames_by_contrast[arbitrary_contrast])
        for bindex in tqdm(range(0, num_images, batch_size), total=num_images):
            filenames = filenames_by_contrast[arbitrary_contrast][bindex:bindex + batch_size]
            for findex, filename in enumerate(filenames):
                for cindex, contrast in enumerate(contrasts):
                    # load raw image batches and normalize the pixels
                    tmp_img = nib.load(filename.replace(arbitrary_contrast, contrast)).get_fdata()
                    try:
                        tmp_img = scaler.fit_transform(tmp_img.reshape(-1, tmp_img.shape[-1])).reshape(tmp_img.shape)
                    except:
                        print(filename)
                        print(contrast)
                    x_batch[findex, ..., cindex] = crop_3D(tmp_img, shape)
                    # load mask batches and change to categorical
                    tmp_mask = nib.load(filename.replace(arbitrary_contrast, 'seg')).get_fdata()
                    tmp_mask[tmp_mask==4] = 3
                    tmp_mask = crop_3D(tmp_mask, crop_size)
                    tmp_mask = to_categorical(tmp_mask, num_classes = 4)
                    y_batch[findex] = tmp_mask

            if bindex + batch_size > num_images:
                x_batch, y_batch = x_batch[:num_images - bindex], y_batch[:num_images - bindex]
            if augment_size is not None:
                # x_aug, y_aug = augment(x_batch, y_batch, augment_size)
                x_aug = None
                y_aug = None
                yield np.append(x_batch, x_aug), np.append(y_batch, y_aug)
            else:
                yield x_batch, y_batch
        if not infinite:
            break


In [5]:
tumours = ['LGG','HGG']

# prefix = '/Users/jasonfung/Documents/EECE571' # Jason's Macbook
prefix = 'C:/Users/Fungj/Documents/EECE_571F' # Jason's Desktop
brats_dir = '/MICCAI_BraTS_2018_Data_Training/'
# prefix = '/home/atom/Documents/datasets/brats' # Adam's Station
file_pattern = '{prefix}/MICCAI_BraTS_2018_Data_Training/{tumour}/{patient_id}/{patient_id}_{contrast}.nii.gz'
# patient_id = 'Brats18_TCIA09_620_1'
contrasts = ['t1ce', 'flair', 't2']
tumours = ['LGG', 'HGG']

data_list_LGG = os.listdir(os.path.join(prefix+brats_dir,tumours[0]))
data_list_HGG = os.listdir(os.path.join(prefix+brats_dir,tumours[1]))
dataset_file_list = data_list_HGG + data_list_LGG

# shuffle and split the dataset file list
import random
random.seed(42)
file_list_shuffled = dataset_file_list.copy()
random.shuffle(file_list_shuffled)
test_ratio = 0.2

train_file, test_file = file_list_shuffled[0:int(len(file_list_shuffled)*(1-test_ratio))], file_list_shuffled[int(len(file_list_shuffled)*(1-test_ratio)):]

while '.DS_Store' in train_file:
    train_file.remove('.DS_Store')
while '.DS_Store' in test_file:
    test_file.remove('.DS_Store')

In [6]:
from keras.metrics import MeanIoU
from tensorflow.keras.utils import to_categorical

batch_size = 1
test_datagen = test_generate_brats_batch(file_pattern, contrasts, batch_size = batch_size, patient_ids = test_file, crop_size= (128,128,128)) # first iteration

# predict on generator
n_classes = 4
IOU_keras = MeanIoU(num_classes=n_classes)



i = 0

while i < len(test_file)//batch_size:
    i += 1
    test_image_batch, test_mask_batch = test_datagen.__next__()
    
    test_mask_batch_argmax = np.argmax(test_mask_batch, axis=4)
    test_pred_batch = my_model.predict(test_image_batch)
    test_pred_batch_argmax = np.argmax(test_pred_batch, axis=4)
    IOU_keras.reset_state()
    IOU_keras.update_state(test_pred_batch_argmax, test_mask_batch_argmax)
    print("Mean IoU =", IOU_keras.result().numpy())


  2%|█▍                                                                                 | 1/57 [00:06<06:18,  6.75s/it]

Mean IoU = 0.4278142


  4%|██▉                                                                                | 2/57 [00:08<03:20,  3.64s/it]

Mean IoU = 0.70331657


  5%|████▎                                                                              | 3/57 [00:09<02:24,  2.67s/it]

...

Mean IoU = 0.3220101


 96%|███████████████████████████████████████████████████████████████████████████████   | 55/57 [01:29<00:03,  1.50s/it]

Mean IoU = 0.49507034


 98%|████████████████████████████████████████████████████████████████████████████████▌ | 56/57 [01:31<00:01,  1.55s/it]

Mean IoU = 0.48433536
Mean IoU = 1.0


In [6]:
print(test_file)

['Brats18_CBICA_AQG_1', 'Brats18_TCIA02_321_1', 'Brats18_CBICA_AXM_1', 'Brats18_TCIA09_141_1', 'Brats18_TCIA09_428_1', 'Brats18_TCIA02_473_1', 'Brats18_TCIA02_605_1', 'Brats18_2013_16_1', 'Brats18_CBICA_ATB_1', 'Brats18_TCIA02_274_1', 'Brats18_CBICA_AAB_1', 'Brats18_CBICA_AXW_1', 'Brats18_TCIA10_202_1', 'Brats18_CBICA_ALU_1', 'Brats18_TCIA02_198_1', 'Brats18_TCIA01_231_1', 'Brats18_TCIA05_444_1', 'Brats18_2013_23_1', 'Brats18_TCIA08_319_1', 'Brats18_CBICA_ASH_1', 'Brats18_TCIA02_394_1', 'Brats18_CBICA_AWH_1', 'Brats18_TCIA10_632_1', 'Brats18_CBICA_AXL_1', 'Brats18_CBICA_ABB_1', 'Brats18_CBICA_AYA_1', 'Brats18_CBICA_AAP_1', 'Brats18_TCIA13_633_1', 'Brats18_TCIA03_375_1', 'Brats18_TCIA01_150_1', 'Brats18_CBICA_ATV_1', 'Brats18_TCIA10_640_1', 'Brats18_TCIA03_474_1', 'Brats18_TCIA13_624_1', 'Brats18_CBICA_AUN_1', 'Brats18_2013_13_1', 'Brats18_TCIA02_283_1', 'Brats18_TCIA09_493_1', 'Brats18_TCIA01_186_1', 'Brats18_2013_24_1', 'Brats18_CBICA_AZD_1', 'Brats18_2013_26_1', 'Brats18_TCIA10_625_1

In [11]:
# prefix = '/Users/jasonfung/Documents/EECE571' # Jason's Macbook
# prefix = '/home/atom/Documents/datasets/brats' # Adam's Station
prefix = 'C:/Users/Fungj/Documents/EECE_571F' # Jason's Desktop
brats_dir = '/MICCAI_BraTS_2018_Data_Training/'

contrasts = ['t1ce', 'flair', 't2']

# Get random image
import random

random.seed(42)
img_num = random.randint(0,len(test_file))
name = test_file[img_num]

raw_img = np.empty((batch_size, ) + (128,128,128) + (len(contrasts), ))

if name in os.listdir(prefix+brats_dir+'HGG'):
    # load t2 and preprocess the data
    for cindex, contrast in enumerate(contrasts):
        tmp_img = nib.load(os.path.join(prefix+brats_dir+'HGG'+f'/{name}/{name}_{contrast}.nii.gz')).get_fdata()
        tmp_img = scaler.fit_transform(tmp_img.reshape(-1, tmp_img.shape[-1])).reshape(tmp_img.shape)
        tmp_img = crop_3D(tmp_img,(128,128,128))
        raw_img[0,...,cindex] = tmp_img
    
    ground_truth_mask = np.int_(nib.load(os.path.join(prefix+brats_dir+'HGG'+f'/{name}/{name}_seg.nii.gz')).get_fdata())
    ground_truth_mask[ground_truth_mask==4] = 3
    ground_truth_mask = crop_3D(ground_truth_mask,(128,128,128))
    
else:
    # load t2 and preprocess the data
    for cindex, contrast in enumerate(contrasts):
        tmp_img = nib.load(os.path.join(prefix+brats_dir+'LGG'+f'/{name}/{name}_{contrast}.nii.gz')).get_fdata()
        tmp_img = scaler.fit_transform(tmp_img.reshape(-1, tmp_img.shape[-1])).reshape(tmp_img.shape)
        tmp_img = crop_3D(tmp_img,(128,128,128))
        raw_img[0,...,cindex] = tmp_img
        
    ground_truth_mask = np.int_(nib.load(os.path.join(prefix+brats_dir+'HGG'+f'/{name}/{name}_seg.nii.gz')).get_fdata())
    ground_truth_mask[ground_truth_mask==4] = 3
    ground_truth_mask = crop_3D(ground_truth_mask,(128,128,128))
    

In [12]:
predict_mask = my_model.predict(raw_img) # perform inference
predict_mask = np.argmax(predict_mask, axis=4)[0,:,:,:]

In [13]:
ground_truth_mask.astype(int)
ground_truth_mask.max()

3

# Visualize MRI Volume

In [14]:
import napari
viewer = napari.view_image(raw_img[0,...,0], name = name + '' + contrasts[0])
viewer.add_image(raw_img[0,...,1], name = name + '' + contrasts[1])
viewer.add_image(raw_img[0,...,2], name = name + '' + contrasts[2])

gt_layer = viewer.add_labels(ground_truth_mask, name = "ground truth")
pred_layer = viewer.add_labels(predict_mask, name = "predicted")